In [1]:
import cv2
import numpy as np
from scipy.spatial import distance
import tensorflow.keras.backend as K
import tensorflow.keras.models as Models
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing import image
from sklearn import preprocessing
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pandas as pd
import face_recognition_functions as frf
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import random as rand

# 1 FASE: Reconocimento facial

In [2]:
data_path = 'imagenes'

#Detector de caras basado en redes convolucionales 2D
detector_dnn = cv2.dnn.readNetFromCaffe('deploy.prototxt.txt', 'res10_300x300_ssd_iter_140000.caffemodel')

#CARGAMOS EL MODELO DE RECONOCIMIENTO FACIAL basado en Resnet-50 y entrenado con VGG-Face
model_file = 'resnet50.h5'
model = Models.load_model(model_file)
last_layer = model.get_layer('avg_pool').output
feature_layer = Flatten(name = 'flatten')(last_layer)
feature_extractor = Models.Model(model.input, feature_layer)

## Pruebas del reconocimiento facial

In [3]:
""""Comparamos si son la misma persona a través de una simple.Detectamos las caras de las imagenes. Imagen0 y Imagen1"""

#Ejemplo para dos imágenes
img_face1=frf.extract_faces(data_path+"/0.jpg")
img_face2=frf.extract_faces(data_path+"/1.jpg")

In [4]:
"Comparamos si es la misma persona"

embedding = frf.generate_embedding(img_face1)
embedding = np.asarray(embedding)
f1 = np.squeeze(embedding)
embedding = frf.generate_embedding(img_face2)
embedding = np.asarray(embedding)
f2 = np.squeeze(embedding)

1/1 [==============================] - 0s 151ms/step


In [ ]:
N = 10
score_genuino = []
score_impostor = []

frf.comparation(0,1,data_path)

In [ ]:
""" Bucle para comparar """

index_jpg = [i for i in range(N)]
print(index_jpg)
for i in range(N):
    for j in index_jpg:
        if i!=j:
            sim=frf.comparation(i,j, data_path)
            if sim>=0.50:
                print("Misma persona",i,"y",j)
                score_genuino.append(sim)
            else:
                print("Diferentes personas",i,"y",j)
                score_impostor.append(sim)
    index_jpg.remove(i)
    print(index_jpg)



In [ ]:
# Gráfica de FRR vs FAR
sorted_genuino = sorted(score_genuino)
sorted_impostor = sorted(score_impostor)

print(sorted_genuino)
print(sorted_impostor)

ax = pd.DataFrame(sorted_genuino).plot(kind='density')
pd.DataFrame(sorted_impostor).plot(ax=ax,kind='density')
ax.axvline(0.434, color='g', linestyle='--')
ax.legend(["Genuines Distribution","Impostors Distribution"])
ax.text(0.38,-0.3,"FRR")
ax.text(0.45,-0.3,"FAR")
plt.xlabel("Matching score value")

plt.show()

# 2 FASE: Embedings. Crear vectores que representen las imagenes a utilizar

In [3]:
def obtain_data_embedding(v_carpeta, n_identidades, n_archivos):
    ##estamos cogiendo los n_archivos primeros, conviene tomar los archivos de forma aleatoria??
    X=[] # lista de embedings
    y=[] # lista de etiquetas 
    
    for carpeta in v_carpeta:
        aux = 0
        directorio = "4K_120\\" + carpeta
        print(directorio)
        # Recorre de manera recursiva todos los archivos y carpetas en la ruta especificada
        for ruta, carpetas, archivos in os.walk(directorio):
            # Recorre todos los archivos en la carpeta actual
            for archivo in archivos[0:n_archivos]:
                print(carpetas)
                # Genera la ruta completa del archivo
                ruta_archivo = os.path.join(ruta, archivo)
                img =  cv2.imread(ruta_archivo, cv2.IMREAD_UNCHANGED)

                #Extraee los embeddings del archivo
                embeddings = frf.generate_embedding(img)

                #Agrega los embeddings a la lista 
                X.append(embeddings[0])

                #Agregamos la etiqueta a la lista
                if carpeta[0] == 'H':
                    y.append(0)
                else:
                    y.append(1)

                # Parar bucle 
                aux = aux +1
                if aux>=n_identidades:
                    break
            else:
                continue
            break
    
    X = np.array(X)
    return X,y

def representacion_TSNE(X, n_components = 2, perplexity = 1):
    X_embedded = TSNE(n_components=n_components, learning_rate='auto',init='random',perplexity=perplexity).fit_transform(X)
    #X_embedded.reshape(n_identidades, )
    
    #TO DO: ploteo de los embeddings
    return X_embedded


# FASE 3: Clasificación

In [4]:
# Se crea un modelo para el clasificador
input_size = 2048
def baseline_model():
    model = Sequential()
    model.add(Dense(60, input_dim=2048, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Se crea el clasificador 
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

<ipython-input-4-1d6c6a249782>:11: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)


# EJECUCIÓN
Entrenamiento con asiáticos y predicción en caucásicos.

In [5]:
## EMBEDING
n_identidades = 500
# Hombres
x_HA, y_HA = obtain_data_embedding(["HA4K_120"],n_identidades,1) # asiáticos
x_HB, y_HB = obtain_data_embedding(["HB4K_120"],n_identidades,1) # blancos
x_HN, y_HN = obtain_data_embedding(["HN4K_120"],n_identidades,1) # negros
# Mujeres
x_MA, y_MA = obtain_data_embedding(["MA4K_120"],n_identidades,1) # asiáticos
x_MB, y_MB = obtain_data_embedding(["MB4K_120"],n_identidades,1) # blancos
x_MN, y_MN = obtain_data_embedding(["MN4K_120"],n_identidades,1) # negros

## DATOS DE ENTRENAMIENTO
index_train = int(70*(n_identidades)/100) # 70% de los datos
x_A = x_HA[0:index_train] + x_MA[0:index_train]
x_train = np.array(x_A).astype(float)
y_train = np.asarray(y_A)
y_train = np_utils.to_categorical(y_train)

# Datos de test
x_test = np.array(x_B).astype(float)
y_test = np.asarray(y_B)
y_test = np_utils.to_categorical(y_test)

# Entrenamiento del clasificador
estimator.fit(x_train, y_train)

# Predicción 
y_pred = estimator.predict(x_test)
y_pred = np_utils.to_categorical(y_pred)

# Resultados
score = accuracy_score(y_test, y_pred)
print("El score es el siguiente valor",score)

4K_120\HA4K_120
[]
1/1 [==============================] - 1s 1s/step
[]
1/1 [==============================] - 0s 113ms/step
[]
1/1 [==============================] - 0s 121ms/step
[]
1/1 [==============================] - 0s 116ms/step
[]
1/1 [==============================] - 0s 113ms/step
[]
1/1 [==============================] - 0s 139ms/step
[]
1/1 [==============================] - 0s 113ms/step
[]
1/1 [==============================] - 0s 110ms/step
[]
1/1 [==============================] - 0s 112ms/step
[]
1/1 [==============================] - 0s 131ms/step
[]
1/1 [==============================] - 0s 113ms/step
[]
1/1 [==============================] - 0s 115ms/step
[]
1/1 [==============================] - 0s 113ms/step
[]
1/1 [==============================] - 0s 103ms/step
[]
1/1 [==============================] - 0s 113ms/step
[]
1/1 [==============================] - 0s 108ms/step
[]
1/1 [==============================] - 0s 110ms/step
[]
1/1 [===========================